## Verilerin regresyon kullanılarak açıklanması

Gerekli paketlerin yüklenmesi

In [10]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import cross_val_predict
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.linear_model import Lasso, Ridge, ElasticNet

Verilerin yüklenmesi ve değişken tiplerinin ayarlanması

In [5]:
df = pd.read_csv('hepsiemlak_cleaned.csv')

In [6]:
df['city'] = df['city'].astype('category')
df['district'] = df['district'].astype('category')
df['neighborhood'] = df['neighborhood'].astype('category')
df['room'] = df['room'].astype('int')
df['living_room'] = df['living_room'].astype('int')
df['area'] = df['area'].astype('int')
df['age'] = df['age'].astype('int')
df['floor'] = df['floor'].astype('int')
df['price'] = df['price'].astype('int')

print(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6116 entries, 0 to 6115
Data columns (total 9 columns):
 #   Column        Non-Null Count  Dtype   
---  ------        --------------  -----   
 0   city          6116 non-null   category
 1   district      6116 non-null   category
 2   neighborhood  6116 non-null   category
 3   room          6116 non-null   int64   
 4   living_room   6116 non-null   int64   
 5   area          6116 non-null   int64   
 6   age           6116 non-null   int64   
 7   floor         6116 non-null   int64   
 8   price         6116 non-null   int64   
dtypes: category(3), int64(6)
memory usage: 334.8 KB
None


Nitel ve nicel değişkenlerin ayrılması

In [3]:
categorical_features = ['city', 'district', 'neighborhood']
numerical_features = ['room', 'living_room', 'area', 'age', 'floor']

In [4]:
full_pipeline = ColumnTransformer([
    ('num', StandardScaler(), numerical_features),
    ('cat', OneHotEncoder(handle_unknown='ignore'), categorical_features)
])

X matrisi ve y vektörlerinin belirlenmesi

In [7]:
X = df.drop('price', axis=1)
y = df['price']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Regresyon modelinin oluşturulması ve fit işleminin gerçekleştirilmesi

In [8]:
model = Pipeline([
    ('preparation', full_pipeline),
    ('model', LinearRegression())
])

model.fit(X_train, y_train)

Pipeline(steps=[('preparation',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['room', 'living_room',
                                                   'area', 'age', 'floor']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['city', 'district',
                                                   'neighborhood'])])),
                ('model', LinearRegression())])

Modelin değerlendirilmesi

In [9]:
y_pred = model.predict(X_test)
mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
r2 = r2_score(y_test, y_pred)

print(f"MSE: {mse}")
print(f"RMSE: {rmse}")
print(f"R2: {r2}")

MSE: 48486462.418753415
RMSE: 6963.2221290687985
R2: 0.5864233297232997


Değişkenlerin önemlerinin gösterilmesi

In [12]:
if hasattr(model.named_steps['model'], 'coef_'):
    feature_importances = model.named_steps['model'].coef_
    print('Linear Regression Feature Importances')
    print('Intercept:', feature_importances[0])
    print()
    print('Numerical Features')
    for i in range(len(numerical_features)):
        print(numerical_features[i], feature_importances[i])
    print()
    print('Categorical Features')
    for i in range(len(categorical_features)):
        for j in range(len(model.named_steps['preparation'].transformers_[1][1].categories_[i])):
            print(model.named_steps['preparation'].transformers_[1][1].categories_[i][j], feature_importances[len(numerical_features) + j])

Linear Regression Feature Importances
Intercept: 798.9555016405949

Numerical Features
room 798.9555016405949
living_room 0.0
area 3451.287776316445
age -2067.741075799603
floor 63.66795050040968

Categorical Features
afyonkarahisar -5564.319689840116
aydin 1747.6369153185326
denizli -3451.5008829216204
izmir 6298.213113457973
manisa -1759.2249699742406
mugla 2729.1955137451314
acipayam -5564.319689840116
akhisar 1747.6369153185326
alasehir -3451.5008829216204
aliaga 6298.213113457973
balcova -1759.2249699742406
bayindir 2729.1955137451314
bayrakli -1215.5847727209425
bergama -497.0349046815744
bodrum -2550.90270344476
bolvadin 1863.4463065771727
bornova 6529.231910699042
buca -7410.427456266292
buharkent -3771.0927152556274
cardak -4922.422634714548
cay 16628.267757224377
cesme -558.2859687366633
cigli -2014.286420483873
cine -2436.4928111312797
civril -1754.8535573895565
dalaman -1382.3117373766368
datca -5982.054157742294
didim 15361.701453853866
dikili -4762.571587931666
efeler -17

Örnek bir tahmin

In [18]:
print(df[(df['city'] == 'manisa') & (df['district'] == 'yunusemre') & (df['neighborhood'] == 'guzelyurt')])

        city   district neighborhood  room  living_room  area  age  floor  \
5151  manisa  yunusemre    guzelyurt     1            1    65   13      5   
5198  manisa  yunusemre    guzelyurt     2            1    85    2      3   
5222  manisa  yunusemre    guzelyurt     4            1   196    5      1   
5239  manisa  yunusemre    guzelyurt     1            1    60   11      5   

      price  
5151  15000  
5198  15000  
5222  36000  
5239  11000  


In [20]:
new_data = pd.DataFrame({
    'city': ['manisa'],
    'district': ['yunusemre'],
    'neighborhood': ['guzelyurt'],
    'room': [3],
    'living_room': [1],
    'area': [120],
    'age': [5],
    'floor': [3]
})

print(model.predict(new_data))

[22916.51884532]


Modelin başarısını arttırmak için düzenlileştirme denemesi

In [21]:
lasso_model = Pipeline([
    ('preparation', full_pipeline),
    ('model', Lasso(alpha=2.0))
])
lasso_model.fit(X_train, y_train)
y_pred_lasso = lasso_model.predict(X_test)
mse_lasso = mean_squared_error(y_test, y_pred_lasso)
rmse_lasso = np.sqrt(mse_lasso)
r2_lasso = r2_score(y_test, y_pred_lasso)
print('Lasso Regression')
print('Mean Squared Error:', mse_lasso)
print('Root Mean Squared Error:', rmse_lasso)
print('R^2 Score:', r2_lasso)

Lasso Regression
Mean Squared Error: 47146891.47323889
Root Mean Squared Error: 6866.359404607284
R^2 Score: 0.5978495147573935


In [22]:
ridge_model = Pipeline([
    ('preparation', full_pipeline),
    ('model', Ridge(alpha=1.5))
])
ridge_model.fit(X_train, y_train)
y_pred_ridge = ridge_model.predict(X_test)
mse_ridge = mean_squared_error(y_test, y_pred_ridge)
rmse_ridge = np.sqrt(mse_ridge)
r2_ridge = r2_score(y_test, y_pred_ridge)
print('Ridge Regression')
print('Mean Squared Error:', mse_ridge)
print('Root Mean Squared Error:', rmse_ridge)
print('R^2 Score:', r2_ridge)

Ridge Regression
Mean Squared Error: 47484559.7620913
Root Mean Squared Error: 6890.9041324118925
R^2 Score: 0.5949692937720902


In [23]:
elasticnet_model = Pipeline([
    ('preparation', full_pipeline),
    ('model', ElasticNet(alpha=0.1, l1_ratio=0.5))
])
elasticnet_model.fit(X_train, y_train)
y_pred_elasticnet = elasticnet_model.predict(X_test)
mse_elasticnet = mean_squared_error(y_test, y_pred_elasticnet)
rmse_elasticnet = np.sqrt(mse_elasticnet)
r2_elasticnet = r2_score(y_test, y_pred_elasticnet)
print('ElasticNet Regression')
print('Mean Squared Error:', mse_elasticnet)
print('Root Mean Squared Error:', rmse_elasticnet)
print('R^2 Score:', r2_elasticnet)

ElasticNet Regression
Mean Squared Error: 65188362.643805474
Root Mean Squared Error: 8073.931052703229
R^2 Score: 0.44396054861310186


Lineer olmayan ilişkileri açıklayabilmek için diğer regresyon modellerinin denenmesi

In [24]:
random_forest_model = Pipeline([
    ('preparation', full_pipeline),
    ('model', RandomForestRegressor(random_state=42))
])
random_forest_model.fit(X_train, y_train)
y_pred_random_forest = random_forest_model.predict(X_test)
mse_random_forest = mean_squared_error(y_test, y_pred_random_forest)
rmse_random_forest = np.sqrt(mse_random_forest)
r2_random_forest = r2_score(y_test, y_pred_random_forest)
print('Random Forest Regression')
print('Mean Squared Error:', mse_random_forest)
print('Root Mean Squared Error:', rmse_random_forest)
print('R^2 Score:', r2_random_forest)

Random Forest Regression
Mean Squared Error: 49930359.18351838
Root Mean Squared Error: 7066.141746633617
R^2 Score: 0.5741072731086223


In [25]:
gradient_boosting_model = Pipeline([
    ('preparation', full_pipeline),
    ('model', GradientBoostingRegressor(random_state=42))
])
gradient_boosting_model.fit(X_train, y_train)
y_pred_gradient_boosting = gradient_boosting_model.predict(X_test)
mse_gradient_boosting = mean_squared_error(y_test, y_pred_gradient_boosting)
rmse_gradient_boosting = np.sqrt(mse_gradient_boosting)
r2_gradient_boosting = r2_score(y_test, y_pred_gradient_boosting)
print('Gradient Boosting Regression')
print('Mean Squared Error:', mse_gradient_boosting)
print('Root Mean Squared Error:', rmse_gradient_boosting)
print('R^2 Score:', r2_gradient_boosting)

Gradient Boosting Regression
Mean Squared Error: 55575004.35926098
Root Mean Squared Error: 7454.8644762504555
R^2 Score: 0.5259599462008515


In [26]:
support_vector_model = Pipeline([
    ('preparation', full_pipeline),
    ('model', SVR(kernel='linear'))
])
support_vector_model.fit(X_train, y_train)
y_pred_support_vector = support_vector_model.predict(X_test)
mse_support_vector = mean_squared_error(y_test, y_pred_support_vector)
rmse_support_vector = np.sqrt(mse_support_vector)
r2_support_vector = r2_score(y_test, y_pred_support_vector)
print('Support Vector Regression')
print('Mean Squared Error:', mse_support_vector)
print('Root Mean Squared Error:', rmse_support_vector)
print('R^2 Score:', r2_support_vector)

Support Vector Regression
Mean Squared Error: 115736483.20859876
Root Mean Squared Error: 10758.089198765678
R^2 Score: 0.012798480913012278


Çapraz geçerleme ve hiper parametre analizi ile model başarısını arttırma denemesi

In [27]:
param_grid = [
    {'model__n_estimators': [3, 10, 30, 50, 100, 200, 500, 1000], 'model__max_features': [2, 4, 6, 8, 10, 12]},
    {'model__bootstrap': [False], 'model__n_estimators': [3, 10], 'model__max_features': [2, 3, 4]}
]

In [28]:
grid_search = GridSearchCV(random_forest_model, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(X_train, y_train)
print('Random Forest Regression Best Parameters:', grid_search.best_params_)
print('Random Forest Regression Best Estimator:', grid_search.best_estimator_)
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

y_pred_random_forest_best = grid_search.best_estimator_.predict(X_test)
mse_random_forest_best = mean_squared_error(y_test, y_pred_random_forest_best)
rmse_random_forest_best = np.sqrt(mse_random_forest_best)
r2_random_forest_best = r2_score(y_test, y_pred_random_forest_best)
print('Random Forest Regression Best Estimator')
print('Mean Squared Error:', mse_random_forest_best)
print('Root Mean Squared Error:', rmse_random_forest_best)
print('R^2 Score:', r2_random_forest_best)

Random Forest Regression Best Parameters: {'model__max_features': 12, 'model__n_estimators': 1000}
Random Forest Regression Best Estimator: Pipeline(steps=[('preparation',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['room', 'living_room',
                                                   'area', 'age', 'floor']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['city', 'district',
                                                   'neighborhood'])])),
                ('model',
                 RandomForestRegressor(max_features=12, n_estimators=1000,
                                       random_state=42))])
7554.927382593036 {'model__max_features': 2, 'model__n_estimators': 3}
6751.66510467025 {'model__max_features': 2, 'model__n_estimators': 10}
6552.68532

In [29]:
param_grid = [
    {'model__n_estimators': [3, 10, 30, 50, 100, 200, 500, 1000], 'model__learning_rate': [0.01, 0.03, 0.1, 0.3, 1.0], 'model__max_depth': [2, 3, 4, 5, 6, 7, 8]}
]
grid_search = GridSearchCV(gradient_boosting_model, param_grid, cv=5, scoring='neg_mean_squared_error', return_train_score=True)
grid_search.fit(X_train, y_train)
print('Gradient Boosting Regression Best Parameters:', grid_search.best_params_)
print('Gradient Boosting Regression Best Estimator:', grid_search.best_estimator_)
cvres = grid_search.cv_results_
for mean_score, params in zip(cvres['mean_test_score'], cvres['params']):
    print(np.sqrt(-mean_score), params)

y_pred_gradient_boosting_best = grid_search.best_estimator_.predict(X_test)
mse_gradient_boosting_best = mean_squared_error(y_test, y_pred_gradient_boosting_best)
rmse_gradient_boosting_best = np.sqrt(mse_gradient_boosting_best)
r2_gradient_boosting_best = r2_score(y_test, y_pred_gradient_boosting_best)
print('Gradient Boosting Regression Best Estimator')
print('Mean Squared Error:', mse_gradient_boosting_best)
print('Root Mean Squared Error:', rmse_gradient_boosting_best)
print('R^2 Score:', r2_gradient_boosting_best)

Gradient Boosting Regression Best Parameters: {'model__learning_rate': 0.1, 'model__max_depth': 4, 'model__n_estimators': 500}
Gradient Boosting Regression Best Estimator: Pipeline(steps=[('preparation',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['room', 'living_room',
                                                   'area', 'age', 'floor']),
                                                 ('cat',
                                                  OneHotEncoder(handle_unknown='ignore'),
                                                  ['city', 'district',
                                                   'neighborhood'])])),
                ('model',
                 GradientBoostingRegressor(max_depth=4, n_estimators=500,
                                           random_state=42))])
10094.381738364043 {'model__learning_rate': 0.01, 'model__max_depth': 2, 'model__n_estimators': 3}
9893.269975858766 